In [ ]:
from time import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline

import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
from scipy import stats
from scipy.stats import norm, skew #for some statistics
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
from subprocess import check_output
import os


from time import time
tik = time()
tik_local = time()

def mytime(tik_local):
    tak = time()
    print('all time is - ', tak-tik, '(sec)')
    print('local time is - ', tak-tik_local, '(sec)')
    return tak
#procedure start example of def mytime
#tik_local = mytime(tik_local)


#Unique Data Analysis
#процедура аналицирует количество одинаковых значений в столбце и если их больше чем unique_part*100%,
#то создает список на удаление
def unique_analysis(unique_part):
    col_for_drop = []
    for col in all_data.columns:
        if (np.max(all_data[col].value_counts())/len(all_data[col]))>unique_part:
            print(col,np.max(all_data[col].value_counts()), ' of ', len(all_data[col]), ' per cent is ', 
                  np.max(all_data[col].value_counts())*100//len(all_data[col]),'%')
            col_for_drop.append(col)
    print
    return col_for_drop #возвращает список столбцов таблицы с количеством большим чем unique_part
#procedure start example of def unique_analysis
#col_for_drop = unique_analysis(0.8)


#Missing Data analysis
def missing_analysis(all_data):
    all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
    all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
    missing_data = pd.DataFrame({'Missing Ratio (% of total all_data)' :all_data_na})
    print(missing_data.head(20))
#procedure start example of def unique_analysis
#missing_analysis(all_data)


#determinate train to numeric and categoric data
# Take analysis of numeric and categorical columns
def columns_len(all_data):
    numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
    start_len_num = len(numeric_feats)
    print('len of the numeric_feats is: ', start_len_num)
    categorical_feats = all_data.dtypes[all_data.dtypes == "object"].index
    start_len_cat = len(categorical_feats)
    print('len of the categorical_feats is: ', start_len_cat)
    print('total len of all_data_feats is: ', len(all_data.columns))
    return numeric_feats, categorical_feats


#other def for determing the numeric and categiric features (columns of dataframe) as lists
def list_columns(all_data):
    cat_cols = [c for c in all_data.columns if all_data[c].dtype.name == 'object']
    num_cols   = [c for c in all_data.columns if all_data[c].dtype.name != 'object']
    return num_cols, cat_cols
#procedure start example of def list_columns
#num_cols, cat_cols = list_columns(all_data)


# Определение вылетов по межквартильному размаху в столбцах с цифрами
# Помечаем вылеты через дополнительные столбцы col +'Outlier'
#создаем функцию, которая возвращает индекс выбросов
def indicies_of_outliers(x):
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3-q1
    lower_bond = q1 - (iqr*1.5)
    upper_bond = q3 + (iqr*1.5)
    return np.where((x > upper_bond) | (x < lower_bond))
#основная функция
# делаем дополнительные бинарные столбы с признаком выброса по каждому числовому столбцу
def outlier_mark_percentile(all_data):
    num_cols, cat_cols = list_columns(all_data)
    for col in num_cols:
        pp = indicies_of_outliers(all_data[col])
        list_index = pp[0]
        if len(list_index)>1:
            name_col = col +'Outlier'
            all_data[name_col] = 0
            all_data[name_col][list_index] = 1
            print(len(list_index), ' --- ', all_data[name_col].sum(), ' --- ', name_col)
#procedure start example of outlier_mark_percentile
#outlier_mark_percentile(all_data)        



# класстеризация поля '1stFlrSF', '2ndFlrSF' на 5 групп через метод к-средних
# и запись распределение данных по группам в новую колонку
from sklearn.cluster import KMeans
def kmean_clust(dataframe, col1, col2, num_groups):  #dataframe, col1, col2 as string, num_groups as int
    dataframe = dataframe[[col1, col2]]
    #создаем кластеризатор по методу К средних
    clusterer = KMeans(num_groups, random_state = 0)
    #выполнить подгонку кластеризатора
    clusterer.fit(dataframe)
    #предсказать значения в новую колонку
    dataframe[col1+col2+'kmean_clust'] = clusterer.predict(dataframe)


#Снижение размерности с помощью выделения признаков (метод главных компонент)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
def pca_features(all_data):
    num_cols, cat_cols = list_columns(all_data)
    features = StandardScaler().fit_transform(all_data[num_cols])
    pca = PCA(n_components=0.99, whiten = True)
    features_pca = pca.fit_transform(features)
    print('исходное количество признаков:', features.shape[1])
    print('сокращенное количество признаков (pca):', features_pca.shape[1])


#Уменьшение количества признаков путем максимизации разделимости классов
# до количества столбцов n_components, остальные считаются менее важными
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
def cat_features(all_data, target, n_components):
    num_cols, cat_cols = list_columns(all_data)
    features = all_data[num_cols]
    target = target
    lda = LinearDiscriminantAnalysis(n_components =n_components)
    features_lda = lda.fit(features, target).transform(features)
    print('исходное количество признаков:', features.shape[1])
    print('сокращенное количество признаков lda:', features_lda.shape[1])


#уменьшения количества признаков на разряженных матрицах
# надо доделать после one-hot coding на категорийных столбцах 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
def feat_unrise_onehot(all_data, target, n_components):    #n_components as int
    num_cols, cat_cols = list_columns(all_data)
    features = all_data[num_cols]
    target = target
    features = StandardScaler().fit_transform(features)
    feature_sparse = csr_matrix(features)
    tsvd = TruncatedSVD(n_components = n_components)
    features_sparse_tsvd = tsvd.fit(feature_sparse).transform(feature_sparse)
    print('исходное количество признаков:', features.shape[1])
    print('сокращенное количество признаков lda:', features_sparse_tsvd.shape[1])    


#пороговая обработка дисперсии числовых признаков
#в этом методе признаки не должны быть стандартизованы иначе у них у всех будет одинаковая дисперсия
#суть метода в том, что высокодисперсные признаки несут больше информации чем низкодисперсные
from sklearn.feature_selection import VarianceThreshold
def feat_threshold(all_data, target):
    num_cols, cat_cols = list_columns(all_data)
    features = all_data[num_cols]
    target = target
    thresholder = VarianceThreshold(threshold = .5)
    features_high_variance = thresholder.fit_transform(features)
    print('исходное количество признаков:', features.shape[1])
    print('сокращенное количество признаков lda:', features_high_variance.shape[1])
    #print(features_high_variance[:5])
    #взглянуть на дисперсию отобранных признаков
    print(thresholder.fit(features).variances_)


def corr_data(all_data, target):
    num_cols, cat_cols = list_columns(all_data)
    features = all_data[num_cols]
    target = target    
    #обработка высококоррелированных признаков
    #создаем корреляционную матрицу
    corr_matrix = features.corr().abs()
    #выбираем верхний треугольник корреляционной матрицы (т.к. нижний симметричный)
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    #находим индекс столбцов признаком с корреляцией больше 0.95
    to_drop = [column for column in upper.columns if any(upper[column]>0.95)]
    print(to_drop)
    # исключаем эти признаки
    features.drop(to_drop, axis =1).head(3)
    print('количество признаков:', features.shape[1])



#удаление нерелевантных признаков для классификации
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

def kbest_cat(all_data, target):
    num_cols, cat_cols = list_columns(all_data)
    features = all_data[cat_cols]
    target = target
    #for cаtegoric features
    #конвертировать категориальные данные в целые числа
    features = features.astype(int)
    #отобрать два признака с наивысшими значениями 
    #статистического показателя хи-квадрат
    chi2_selector = SelectKBest(chi2, k=2)
    features_kbest = chi2_secetor.fit_transform(features, target)

def kbest_num(all_data, target):
    num_cols, cat_cols = list_columns(all_data)
    features = all_data[num_cols]
    target = target
    #for nuneric features
    #отобрать два признака с наивысшими значениями 
    #статистического показателя F
    fvalue_selector = SelectKBest(f_classif, k=2)
    features_kbest = fvalue_selector.fit_transform(features, target)

def kbest_all(all_data, target):
    #for all types features
    from sklearn.feature_selection import SelectPercentile
    #отобрать верхние 75% признаков с наивысшими значениями 
    #статистического показателя F
    fvalue_selector = SelectPercentile(f_classif, percentile = 75)
    features_kbest = fvalue_selector.fit_transform(all_data, target)

def skewed_features(all_data):
    #Skewed features
    # Check the skew of all numerical features
    skewed_feats = all_data.apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
    print("\nSkew in numerical features: \n")
    skewness = pd.DataFrame({'Skew' :skewed_feats})
    skewness.head(10)
    #Box Cox Transformation of (highly) skewed features
    skewness = skewness[abs(skewness) > 0.75]    # My_DS_work - was 0.75 my vers - 0.5
    print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
    from scipy.special import boxcox1p
    skewed_features = skewness.index
    lam = 0.15
    for feat in skewed_features:
        all_data[feat] = boxcox1p(all_data[feat], lam)
    #all_data[skewed_features] = np.log1p(all_data[skewed_features])
    #Getting dummy categorical features
    all_data = pd.get_dummies(all_data)
    print(all_data.shape)    
    

def rfecv_features(all_data, target, num_ranking):
    #рекурсивное устранение признаков после нормирования признаков через бокс-кокс
    # num_kanking - номер ранжирование признака от самого лучшего (1) до самого плохого, integer
    from sklearn.feature_selection import RFECV
    from sklearn.linear_model import LinearRegression
    import warnings
    warnings.filterwarnings(action='ignore', module = 'scipy', message='^internal gelsd')
    ols = LinearRegression()
    rfecv = RFECV(estimator=ols, step=1, scoring='neg_mean_squared_error')
    rfecv.fit(features, target)
    rfecv.transform(features)
    print(rfecv.n_features_, 'количество наилучших признаков')
    print("какие категории самые лучшие: \n", rfecv.support_)
    print("ранжирование признака от самого лучшего (1) до самого плохого: \n", rfecv.ranking_)
    ii_col = pd.concat([pd.DataFrame(all_data.columns, columns=['feature']), pd.DataFrame(rfecv.ranking_, columns=['ranking'])], axis=1)
    ii_col = ii_col.sort_values(by=['ranking'])
    best_features = ii_col[ii_col['ranking']<num_ranking]['feature']
    #print(ii_col)
    print(best_features)
    return best_features   #as list


# *********************  MODELS **********************************
#neg_mean_squared_error for models
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

#
#procedure start example of outlier_mark_percentile
#score = rmsle_cv(lasso)
#str_lasso = "\nLasso score: {:.4f} ({:.4f})".format(score.mean(), score.std())
#print(str_lasso)


#*********************  Stacking models  *************************************
#Simplest Stacking approach : Averaging base models
#We begin with this simple approach of averaging base models. We build a new class to extend scikit-learn with our 
#model and also to laverage encapsulation and code reuse (inheritance)
#Averaged base models class

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   
#Averaged base models score
#We just average four models here ENet, GBoost, KRR and lasso. Of course we could easily add more models #in the mix.
#****** procedure start example of AveregingModels:
#averaged_models = AveragingModels(models = (ENet, GBoost, model_xgb, lasso))  # was - ENet, GBoost, KRR, lasso
#score = rmsle_cv(averaged_models)
#str_avr_model = " Averaged base models score: {:.4f} ({:.4f})".format(score.mean(), score.std())
#print(str_avr_model)


#***************************  Stacking averaged Models Class   *****************************
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=15):      #was n_folds=5
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

#---------------------------
#Stacking Averaged models Score
#To make the two approaches comparable (by using the same number of models) , we just average Enet KRR 
#and Gboost, then we add lasso as meta-model.
#****** procedure start example of StakingAveregingModels:
#stacked_averaged_models = StackingAveragedModels(base_models = (ENet, ENet, GBoost, KRR), 
#                                                 meta_model = lasso)
#score = rmsle_cv(stacked_averaged_models)
#str_stacking = "Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std())
#print(str_stacking)

"""
#Final Training and Prediction
#StackedRegressor

stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train, stacked_train_pred), ' str_ctackregr_rmsle')

tik_local = mytime(tik_local)

#XGBoost:

model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred), ' str_xgb_rmsle ')

tik_local = mytime(tik_local)

#LightGBM:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred), 'str_lgb_rmsle ')

tik_local = mytime(tik_local)

'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print('stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15')
print(rmsle(y_train,(stacked_train_pred*0.7 +xgb_train_pred*0.15 + lgb_train_pred*0.15 )))

"""

#******************** EXAMPLES OF MODELS **************************
#import libraries
from sklearn.linear_model import BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingClassifier




#LASSO Regression :
#This model may be very sensitive to outliers. So we need to made it more robust on them. For that we 
#use the #sklearn's Robustscaler() method on pipeline
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

#Elastic Net Regression :  again made robust to outliers
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=0.9, random_state=3))

#Kernel Ridge Regression :
from sklearn.kernel_ridge import KernelRidge
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

#Gradient Boosting Regression :  With huber loss that makes it robust to outliers
from sklearn.ensemble import GradientBoostingRegressor
GBoost = GradientBoostingRegressor(loss='huber', learning_rate=0.05, n_estimators=100, subsample=1.0, 
          criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, 
          min_weight_fraction_leaf=0.0, max_depth=4, min_impurity_decrease=0.0, 
          min_impurity_split=None, init=None, random_state=5, max_features='sqrt', 
          alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort='auto', 
          validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)


#DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
model_DTG = DecisionTreeRegressor(criterion='mae', splitter='best', max_depth=4, 
                                  min_samples_split=2, min_samples_leaf=1, 
                                  min_weight_fraction_leaf=0.0, max_features=None, 
                                  random_state=None, max_leaf_nodes=None, 
                                  min_impurity_decrease=0.0, min_impurity_split=None, 
                                  presort=False)


#SGDRegressor :
from sklearn.linear_model import SGDRegressor
model_SGDR = SGDRegressor(loss='squared_loss', penalty='l2', alpha=0.0001, 
                         l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, 
                         random_state=None, learning_rate='invscaling', 
                         eta0=0.01, power_t=0.25, early_stopping=False, 
                         validation_fraction=0.1, n_iter_no_change=5, 
                         warm_start=False, average=False)






